##**SPRINT 02 - Desenvolvimento dos códigos ETL em Python**

O principal objetivo da Sprint 02 foi avançar na implementação do **Data Warehouse**, focando especificamente no desenvolvimento do processo de **ETL (Extração, Transformação e Carregamento)**.

Para atingir esse objetivo, o foco foi a construção dos códigos em Python e configuração do **Apache Airflow** para automatizar e gerenciar as etapas do ETL.

**Os principais passos desta sprint incluiram:**

**Desenvolvimento das Rotinas de Extração:** Foi criado o código Python responsável por extrair os dados das fontes de origem. Isso garantiu que os dados fossem coletados de maneira eficiente e confiável.

**Transformação dos Dados:** Ocorreram as transformações necessárias nos dados extraídos. Isso permitiu incluir limpeza, formatação, agregação e enriquecimento dos dados para análise.

**Carregamento no Data Warehouse:** Após a transformação, os dados foram carregados no Data Warehouse, onde foram armazenados de forma estruturada e organizada. **O Apache Airflow** foi configurado para agendar e executar essas tarefas de carregamento de maneira automatizada e programada.


Ao final desta sprint, o processo de ETL estava completamente implementado e funcionando e permitiu que os dados fossem coletados, transformados e carregados no **Data Warehouse** de forma automatizada e confiável.

##**01- Construindo os códigos ETL com a Linguagem Python**

**Objetivo:**\
Após a modelagem do **Data Warehouse**, a criação das tabelas de dimensões e da tabela Fato, foi construído o processo de **ETL - Extração, Transformação e Carregamento com a linguagem Python e o Apache Airflow.**


###**1.1- Primeiro Bloco de Código " O processo de Imports"**

In [ ]:
# Primeiro Bloco de comandos os "Imports"
import csv
import airflow
import time
import pandas as pd
from datetime import datetime
from datetime import timedelta
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from airflow.operators.postgres_operator import PostgresOperator
from airflow.utils.dates import days_ago


**Comentando os código do Pacote imports**

* **import "CSV"** : Importa o módulo "csv" em Python. Nossa fonte de dados é um arquivo CSV. Portanto, foi necessário manipular esses arquivos em algum momento.

* **import "airflow"** : Importa o pacote "airflow" em Python pois foi necessário construir toda a **DAG (Directed Acyclic Graph, ou Grafo Direcionado Acíclico)**, que foi lida pelo Apache Airflow.

* **import "time"** : Importa o módulo **"time"** em Python. fornece funcionalidades relacionadas ao tempo e à temporização.

* **import pandas as pd** : Importa o módulo **"pandas"** em Python que foi renomeado para **"pd"** para facilitar referências posteriores. O **"pandas"** é uma biblioteca amplamente usada para manipulação e análise de dados. Forneceu estruturas de dados e funções para trabalhar com dados tabulares.

**Em seguida, temos os imports específicos do Apache Airflow:**

* **from datetime import datetime e from datetime import timedelta:**
 Importa as classes **datetime e timedelta** do módulo **datetime** para lidar com datas e horários.

* **from airflow import DAG:** Importa a classe **DAG** do pacote **airflow**, que é fundamental para definir e configurar fluxos de trabalho no Apache Airflow.

* **from airflow.operators.python_operator import PythonOperator:** Importa a classe **PythonOperator** do **pacote airflow.operators.python_operator**, que permite executar tarefas Python como parte de uma DAG.

* **from airflow.operators.postgres_operator import PostgresOperator:** Importa a classe **PostgresOperator do pacote airflow.operators.postgres_operator**, que é usada para executar tarefas relacionadas ao PostgreSQL em uma DAG.

* **from airflow.utils.dates import days_ago:** Importa a função **days_ago** do pacote **airflow.utils.dates**, que é útil para calcular datas com base na contagem regressiva de dias a partir da data atual.

 ---

###**1.2- Segundo Bloco de Código " Criação da DAG - Directed Acyclic Graphs"**
###**Tópico Argumentos**

In [ ]:
# Argumentos
default_args = {
    'owner': 'airflow',
    'start_date': datetime(2023, 1, 1),
    'depends_on_past': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}


**Comentando o código do Argumentos**

**Argumentos:** representa um direcionário de argumentos

* **'owner': 'airflow'** : Indica que o próprietario ou responsável pela DAG é o Airflow.

* **start_date': datetime(2023, 1, 1)** : Indica que a data e a hora que a DAG começará a ser agendada e executada. A DAG começará a ser executada a partir do primeiro dia de janeiro de 2023. A partir desse ponto, a DAG será agendada de acordo com o cronograma especificado, que pode ser definido usando as outras configurações, como **schedule_interval.**

 A data  foi retroativa **datetime(2023, 1, 1)**, pois quando criamos uma GAG, nos não conseguimos mais modificar esta data, dessa forma é sempre bom colocarmos uma data retroativa, para que comecemos a execuação no momento que começarmos a DAG.

* **'depends_on_past': False'** : Indica que a execução atual da DAG não depende do sucesso da execução anterior. Ou seja, cada execução da DAG ocorrerá independentemente do resultado das execuções anteriores.

* **'retries': 1** : Indica que em caso de falha na execução da tarefa, haverá 01 tentativa de retomar a tarefa. Se a tarefa falhar novamente após essa tentativa, não haverá mais retentativas, e a execução será considerada como falha permanente.

* **'retry_delay': timedelta(minutes=5)** : Indica que o intervalo de tempo que deve ser aguardado antes de tentar novamente uma tarefa que falhou. Neste caso, após uma falha, a tarefa será reagendada para uma nova tentativa após 5 minutos. Isso permite um atraso entre as tentativas para evitar sobrecarregar o sistema em caso de falhas frequentes.

---




###**Tópico criação da DAG**

In [ ]:
# Criar a DAG

# https://crontab.guru/
dag_log_solutions = DAG(dag_id = "logsol",
                   default_args = default_args,
                   schedule_interval = '0 0 * * *',
                   dagrun_timeout = timedelta(minutes = 60),
                   description = 'Job ETL de Carga no DW com Airflow',
                   start_date = airflow.utils.dates.days_ago(1)
)


**Comentando o código do Criar DAG**


* **dag_log_solutions = DAG(dag_id = "logsol"** : Indica que a DAG está sendo nomeada como "dag_log_solutions_dsa" com o ID "logsol".

* **default_args = default_args** : Indica a Definição doo argumento e o conteúdo desse argumento que foi a **lista de agumentos definida na etapa anterior.**

* **schedule_interval = '0 0 * * *'** : Indica a Definição do intervalo de agendamento: No caso de "schedule_interval = '0 0 * * *'", isso significa o seguinte:

 "0 0 * * *"é uma expressão cron que define a programação. Ela indica que a DAG será executada diariamente à meia-noite (00:00) todos os dias.
Portanto, a DAG será agendada para ser executada uma vez por dia, sempre à meia-noite, de acordo com esse cronograma.

* **dagrun_timeout = timedelta(minutes = 60)** : Indica a Definição do tempo máximo permitido para a execução de uma instância da DAG (DAG run). Isso significa que cada execução da DAG tem um limite de tempo de 60 minutos (1 hora). Se a execução da DAG não for concluída dentro desse limite de tempo, ela será considerada falha.

* **description = 'Job ETL de Carga no DW com Airflow'** : Indica a Definição de uma descrição ou um resumo do que é o trabalho (job) realizado pela DAG.

* **start_date = airflow.utils.dates.days_ago(1)** : Indica que o start Date será um dia anterior a data que vamos criar no Airflow, ou seja  indica que a DAG começará a ser executada um dia atrás da data atual.
Essa configuração permite agendar a DAG para ser executada a partir de um ponto no passado, o que pode ser útil em cenários em que você deseja retroativamente executar tarefas ou processar dados a partir de uma data anterior.

**Obs:** Temos também um start_date no  default_args, irá gerar conflito? Não gerará conflito e se tivermos o argumentos em dois locais, vale por ultimo o que estiver a DAG

**Dessa forma temos a DAG criada que é o bloco principal para execuação do Apahe Airflow.**

---



###**1.3- Terceiro Bloco de Código " Função para Carregar dados no DW**

In [ ]:
def func_carrega_dados_clientes(**kwargs):

    # Get the csv file path
    csv_file_path = kwargs['params']['csv_file_path']

    # Inicializa o contador
    i = 0

    # Open the csv file
    with open(csv_file_path, 'r') as f:

        reader = csv.DictReader(f)

        for item in reader:

            # Icrementa o contador
            i += 1

            # Extrai uma linha como dicionário
            dados_cli = dict(item)

            # Define as colunas e placeholders para a consulta SQL
            columns = ', '.join(dados_cli.keys())
            placeholders = ', '.join(['%s'] * len(dados_cli))

            # Consulta SQL com placeholders
            sql_query_cli = f"INSERT INTO varejo.DIM_CLIENTE ({columns}) VALUES ({placeholders})"

            # Operador do Postgres com incremento no id da tarefa (para cada linha inserida)
            postgres_operator = PostgresOperator(task_id = 'carrega_dados_clientes_' + str(i),
                                                 sql = sql_query_cli,
                                                 parameters = list(dados_cli.values()),
                                                 postgres_conn_id = 'LOGDW',
                                                 dag = dag_log_solutions)

            # Executa o operador
            postgres_operator.execute(context = kwargs)


tarefa_carrega_dados_clientes = PythonOperator(
        task_id = 'tarefa_carrega_dados_clientes',
        python_callable = func_carrega_dados_clientes,
        provide_context = True,
        op_kwargs = {'params': {'csv_file_path': '/opt/airflow/dags/dados/DIM_CLIENTE.csv'}},
        dag = dag_log_solutions
    )


----

###**1.4- Comentando os tópicos da Função**

Tópico: def func_carrega_dados_clientes(**kwargs):

* **def** : Esta é uma palavra-chave Python usada para definir uma função. É o começo da definição da função.

* **func_carrega_dados_clientes** : Este é o nome da função. É o identificador pelo qual você pode chamar a função posteriormente no seu programa. O nome da função é escolhido pelo programador e deve seguir as regras de nomenclatura do Python.

* (**kwargs)** : Esta é uma parte importante da definição da função e se refere aos parâmetros da função. Vou dividir isso em partes menores:

* **( )** : Os parênteses vazios indicam que a função não aceita parâmetros posicionais, ou seja, não requer argumentos posicionais quando você a chama.

* ****kwargs* : Este é um parâmetro especial que permite que a função aceite um número variável de argumentos nomeados. Aqui estão os detalhes:

****** : É um operador de descompactação que permite que você passe um dicionário de argumentos nomeados para a função.

**kwargs** : É um nome comum usado para se referir aos argumentos nomeados passados para a função. Esses argumentos são empacotados em um dicionário onde as chaves são os nomes dos argumentos e os valores são os valores associados.

---


**Tópico: Get the csv file path ( Obter o caminho do Arquivo CSV)**

In [ ]:
# Get the csv file path
csv_file_path = kwargs['params']['csv_file_path']

* **csv_file_path** : Esta é uma variável que está sendo definida. Ela irá armazenar o valor associado à chave **'csv_file_path'** dentro do dicionário **kwargs**. O nome da variável, neste caso, sugere que ela provavelmente armazenará um caminho de arquivo para um arquivo CSV.

* **kwargs** : É uma convenção comum para nomear o dicionário de argumentos nomeados passados para uma função. Dentro desse dicionário, você pode ter várias chaves e valores que foram passados para a função. Ele permite que você acesse esses valores com base em suas chaves.

* **['params']** : Isso indica que você está acessando um valor específico dentro do dicionário **kwargs** usando a chave **'params'**. Presumivelmente, dentro do dicionário **kwargs**, há uma chave **'params'** que está associada a outro dicionário.

* **['csv_file_path']** : Agora você está acessando uma chave específica dentro do dicionário associado à chave **'params'**. Isso significa que há um dicionário aninhado dentro do dicionário **kwargs** sob a chave **'params'**, e dentro deste dicionário aninhado, você está procurando o valor associado à chave **'csv_file_path'**.

Em resumo, essa linha de código está extraindo o valor associado à chave **'csv_file_path'** de um dicionário aninhado dentro do dicionário **kwargs** e atribuindo esse valor à variável **csv_file_path**. Essa variável provavelmente será usada como o caminho do arquivo para um arquivo CSV em operações posteriores no código.

---

**Tópico: Inicializar o contador**

In [ ]:
# Inicializa o contador
    i = 0

**Nesse código, a linha i = 0 está realizando a seguinte ação:**

* **i** : Isso é uma variável. Em programação, uma variável é usada para armazenar valores, como números, texto ou outros tipos de dados.

* **= 0** : Isso é um operador de atribuição. Ele atribui o valor à direita (neste caso, 0) à variável à esquerda **(neste caso, i)**. Em outras palavras, a variável **i** está sendo inicializada com o **valor 0**.

Portanto, essa linha de código está iniciando a variável **i** e atribuindo o valor **0** a ela. Isso é comum em programação, especialmente em loops e contadores, onde você deseja começar com um valor específico e, em seguida, incrementá-lo ou modificá-lo à medida que o programa é executado. O valor **0** é frequentemente usado como um ponto de partida padrão para muitos contadores em linguagens de programação.

---

**Tópico: Abrir o aqruivo CSV**

In [ ]:
# Open the csv file
    with open(csv_file_path, 'r') as f:

        reader = csv.DictReader(f)

        for item in reader:

* **with open(csv_file_path, 'r') as f** : Esta linha abre o arquivo CSV especificado em csv_file_path no modo de leitura ('r'). A cláusula with é usada para garantir que o arquivo seja corretamente fechado após seu uso, mesmo se ocorrerem exceções durante o processo.

* **reader = csv.DictReader(f)** : Aqui, é criado um objeto reader usando o módulo csv.DictReader. Este objeto é usado para ler o conteúdo do arquivo CSV. O DictReader trata a primeira linha do arquivo CSV como cabeçalho, o que significa que as colunas se tornarão chaves em um dicionário.

* **for item in reader** : Este é um loop for que itera por cada linha do arquivo CSV. Em cada iteração, a variável item contém uma linha de dados do arquivo. A linha é representada como um dicionário, onde as chaves são os nomes das colunas (a partir do cabeçalho) e os valores são os dados nas respectivas colunas.

Com esse código, você pode percorrer todas as linhas do arquivo CSV e acessar os dados de cada linha como um dicionário, o que facilita muito o processamento e a manipulação dos dados.


---

**Tópico: Extrair uma linha como dicionário**

In [ ]:
# Extrai uma linha como dicionário
            dados_cli = dict(item)

* **dados_cli = dict(item)** : Esta linha cria um novo dicionário chamado **dados_cli**. O dicionário é construído a partir do dicionário **item**. A função **dict()** é usada para criar uma cópia do dicionário item. Isso é feito para que possamos acessar facilmente os valores dos campos na linha atual do **arquivo CSV** usando as chaves correspondentes.

No geral, essa linha cria uma cópia dos dados da linha atual do CSV em um dicionário chamado **dados_cli**, permitindo um acesso mais fácil e organizado aos valores dos campos dessa linha.

---

**Tópico: Definir as colunas e placeholders para a consulta SQL**

In [ ]:
# Define as colunas e placeholders para a consulta SQL
            columns = ', '.join(dados_cli.keys())
            placeholders = ', '.join(['%s'] * len(dados_cli))


* **columns = ', '.join(dados_cli.keys())** : Nesta linha, estamos construindo a **string columns**, que conterá uma lista de nomes de colunas separados por vírgulas. **O método dados_cli.keys()** retorna uma lista das chaves **(nomes de colunas)** no **dicionário dados_cli**. A função **join()** é usada para concatenar essas chaves **(nomes de colunas)** em uma única **string**, separando-as por vírgulas. O resultado será algo como **"Nome, Idade, Cidade"**, onde as vírgulas separam os nomes das colunas.

* **placeholders = ', '.join(['%s'] * len(dados_cli))** : Nesta linha, estamos construindo a string placeholders, que conterá uma lista de marcadores de posição **("%s")** separados por vírgulas. A quantidade de marcadores de posição é determinada pelo número de elementos no **dicionário dados_cli.** Para fazer isso, primeiro criamos uma lista contendo **%s** repetido o número de vezes igual ao tamanho do dicionário **dados_cli** usando a lista de compreensão. Em seguida, usamos a função **join()** para concatenar esses marcadores de posição em uma única string, separados por vírgulas.

No geral, essas linhas são frequentemente usadas em consultas SQL para inserção de dados em um banco de dados. A primeira linha **(columns)** cria a parte da consulta SQL que lista os nomes das colunas nas quais os dados serão inseridos, enquanto a segunda linha **(placeholders)** cria a parte da consulta que contém marcadores de posição para os valores que serão inseridos nessas colunas. A combinação dessas duas strings pode ser usada em uma consulta SQL para inserção de dados em um banco de dados.

----

**Tópico: Consultar SQL com placeholders**

In [ ]:
sql_query_cli = f"INSERT INTO varejo.DIM_CLIENTE ({columns}) VALUES ({placeholders})"

Neste trecho de código, estamos construindo uma consulta SQL para inserir dados em uma tabela chamada "varejo.DIM_CLIENTE". Vou explicar o que cada elemento faz:

* **sql_query_cli** : Esta é uma variável que armazenará a consulta SQL completa que será usada para inserir os dados na tabela.

* **f"INSERT INTO varejo.DIM_CLIENTE ({columns}) VALUES ({placeholders})"** : Esta é uma **f-string**, uma forma de criar strings formatadas em Python. Aqui está o que cada parte faz:

**"INSERT INTO varejo.DIM_CLIENTE"** : Esta é a parte da consulta SQL que indica que estamos realizando uma operação de inserção de dados na tabela **"varejo.DIM_CLIENTE"**.

**({columns})** : Isso é uma substituição, onde **{columns}** será substituído pela string que foi construída anteriormente e contém os nomes das colunas na tabela. Por exemplo, se columns for **"Nome, Idade, Cidade"**, a consulta SQL ficará assim: **"INSERT INTO varejo.DIM_CLIENTE (Nome, Idade, Cidade)"**.

**"VALUES"** : Esta parte da consulta SQL indica que estamos fornecendo os valores que serão inseridos na tabela.

**({placeholders})** : Isso é outra substituição, onde {placeholders} será substituído pela string que contém marcadores de posição para os valores que serão inseridos. Por exemplo, se placeholders **for "%s, %s, %s"** (com base na quantidade de colunas e valores), a consulta SQL ficará assim: **"VALUES (%s, %s, %s)"**.

---

**Tópico: Operador do Postgres com incremento no id da tarefa (para cada linha inserida)**

In [ ]:
# Operador do Postgres com incremento no id da tarefa (para cada linha inserida)
            postgres_operator = PostgresOperator(task_id = 'carrega_dados_clientes_' + str(i),
                                                 sql = sql_query_cli,
                                                 parameters = list(dados_cli.values()),
                                                 postgres_conn_id = 'LOGDW',
                                                 dag = dag_log_solutions)

Neste trecho de código, foi configurando um **operador PostgreSQL** no Apache Airflow para executar uma consulta SQL e inserir dados em uma tabela. Vou explicar o que cada elemento faz:

* **postgres_operator** : Esta é uma variável que representa o operador PostgreSQL que será configurado.

* **PostgresOperator** : Isso cria uma instância do operador PostgreSQL no Apache Airflow.

* **task_id** : É um identificador único para a tarefa dentro do fluxo (DAG). O identificador é gerado dinamicamente com base no valor de i. Isso é feito para que cada linha de dados seja inserida em uma tarefa separada, com um identificador único.

* **sql** : É a consulta SQL que será executada pelo operador PostgreSQL. No caso, essa consulta é a que construímos anteriormente usando as variáveis columns e placeholders.

* **parameters** : Esta é uma lista de parâmetros que serão usados para preencher os marcadores de posição na consulta SQL. Os valores da lista são extraídos do dicionário dados_cli usando o método .values(). Isso garante que os valores sejam inseridos nas colunas correspondentes de acordo com a consulta SQL.

* **postgres_conn_id** : Este é o identificador da conexão PostgreSQL que o Apache Airflow usará para se conectar ao banco de dados. A conexão com o nome 'LOGDW' deve ser configurada antecipadamente no Airflow e conter as informações de conexão necessárias, como host, porta, nome do banco de dados, usuário e senha.

* **dag** : É o DAG (Directed Acyclic Graph) ao qual esta tarefa pertence. O operador é adicionado a este DAG para que o Apache Airflow saiba como organizar e executar as tarefas em um fluxo.

Portanto, esse trecho de código configura um **operador PostgreSQL** que executa a consulta **SQL sql_query_cli** com os parâmetros parameters no banco de dados configurado com a conexão **'LOGDW'**.

Cada vez que esse operador é executado **(por exemplo, durante a execução do DAG)**, ele insere uma linha de dados na tabela do banco de dados. **O task_id** é incrementado para que cada linha de dados seja inserida em uma tarefa separada com um nome exclusivo, como **'carrega_dados_clientes_0'**, **'carrega_dados_clientes_1'** e assim por diante.

-----

**Tópico: Executar o Operador**

In [ ]:
# Executa o operador
            postgres_operator.execute(context = kwargs)

Neste trecho de código, estamos chamando o método **execute do operador PostgreSQL configurado anteriormente**.

* **postgres_operator** : Este é o operador PostgreSQL que configuramos anteriormente. O método execute será chamado nesse operador para executar a tarefa.

* **.execute(context=kwargs)**: O método execute é chamado com um único argumento chamado context, que é igual a **kwargs**.

* **context** : O contexto é um dicionário que contém informações e variáveis relacionadas à execução da tarefa. É uma prática comum no Apache Airflow passar o contexto para as tarefas para que elas possam acessar informações sobre a execução atual, como a data de execução, parâmetros, variáveis definidas e muito mais.

* **kwargs** : kwargs é uma abreviação de **"keyword arguments"** e é frequentemente usado para passar argumentos nomeados em Python. Nesse contexto, kwargs representa o dicionário de contexto que contém informações relevantes para a execução da tarefa.

Portanto, o código **postgres_operator.execute(context=kwargs)** está iniciando a execução do **operador PostgreSQL** com base no contexto fornecido em **kwargs**. Isso é importante porque o contexto pode conter informações necessárias para a execução da consulta SQL, como os parâmetros e outras variáveis relevantes para a tarefa.

Em resumo, este código inicia a execução da tarefa do **operador PostgreSQL**, usando as informações no contexto **kwargs** para preencher os detalhes da execução da tarefa, como a data de execução, parâmetros e outros dados relevantes.

---

**Tópico: tarefa_carrega_dados_clientes = PythonOperator**

In [ ]:
tarefa_carrega_dados_clientes = PythonOperator(
        task_id = 'tarefa_carrega_dados_clientes',
        python_callable = func_carrega_dados_clientes,
        provide_context = True,
        op_kwargs = {'params': {'csv_file_path': '/opt/airflow/dags/dados/DIM_CLIENTE.csv'}},
        dag = dag_log_solutions
    )


Este código criou uma tarefa no **Apache Airflow** usando o **operador PythonOperator** para executar uma **função Python**.

* **tarefa_carrega_dados_clientes** : Este é o nome da tarefa. É uma instância do **operador PythonOperator** que será adicionada ao **DAG (grafo acíclico direcionado)** chamado **dag_log_solutions**.

* **task_id** : Define o identificador exclusivo para essa tarefa no contexto do DAG. Neste caso, o identificador é **'tarefa_carrega_dados_clientes'**, que é o nome da tarefa.

* **python_callable** : Isso define a função Python que será executada quando a tarefa for acionada. No código, ele se refere à função **func_carrega_dados_clientes**, que contém a lógica que você deseja executar. Essa função será chamada quando a tarefa for executada.

* **provide_context = True** : Isso configura a tarefa para passar o contexto do Airflow para a **função func_carrega_dados_clientes**. O contexto contém informações sobre a execução da tarefa, como a data da execução, parâmetros, variáveis definidas e muito mais. Ao definir provide_context como True, a **função func_carrega_dados_clientes** pode acessar esse contexto.

* **op_kwargs** : Este é um dicionário que permite passar parâmetros para a função func_carrega_dados_clientes. No código, você está passando um parâmetro chamado 'params' que é outro dicionário. Esse dicionário contém o caminho do arquivo CSV que a função usará **('csv_file_path')**.

* **dag**: Isso especifica o **DAG (grafo acíclico direcionado)** ao qual a tarefa pertence. No caso, a tarefa faz parte do DAG chamado 'dag_log_solutions'.

Portanto, este código criou uma tarefa chamada **'tarefa_carrega_dados_clientes'** que executará a função **func_carrega_dados_clientes** e fornecerá o contexto do Airflow para essa função. A função receberá parâmetros definidos em **op_kwargs**, como o caminho do **arquivo CSV**. Quando o DAG for executado, esta tarefa será acionada, o que executará a **função func_carrega_dados_clientes** com os parâmetros e o contexto apropriados.

---





###**1.5- Observação Importante**:
Os dados que vão compor a tabela **Fatos** e as dimensções abaixo
* **Transportadoras**;
* **Depósitos**;
* **Entregas**;
* **Frete**;
* **Pagamentos**
* **Data**

Passaram pelo mesmo processo acima de criação dos codigos, ou seja o mesmo processo de criação do processo ETL - Extração, Transformação e carregamento desses dados que foi submedido os dados que vão compor a tebala cliente do DW, foram aplicados aos dados das demais dimensões e Fato que serão carregados no DW.

---

**Tópico: limpar as tabelas Antes do Carregamento dos Dados**

Os códigos abaixo foram construidos para executar operações de limpeza em tabelas específicas antes do carregamento de novos dados nessas tabelas do DW.

Essa é uma prática comum em processos de **ETL (Extração, Transformação e Carregamento)** para garantir que as tabelas de destino estejam vazias ou em um estado limpo antes de receberem novos dados. **Isso ajuda a evitar conflitos ou duplicatas durante o carregamento de dados.**

**Limpando a Tabela Fato**

In [ ]:
tarefa_trunca_tb_fato = PostgresOperator(task_id = 'tarefa_trunca_tb_fato', postgres_conn_id = 'LOGDW',
sql = "TRUNCATE TABLE varejo.TB_FATO CASCADE", dag = dag_log_solutions)

* **tarefa_trunca_tb_fato**: Isso é uma variável que representa a tarefa que está sendo definida. O nome **tarefa_trunca_tb_fato** é um identificador único para essa tarefa dentro do **DAG (Directed Acyclic Graph)**. Você usará esse identificador para referenciar e controlar essa tarefa em outras partes do código.

* **task_id** : É um parâmetro obrigatório e é o identificador único da tarefa dentro do DAG. No exemplo, o task_id é definido como 'tarefa_trunca_tb_fato', o que significa que esta tarefa será conhecida pelo nome 'tarefa_trunca_tb_fato'.

* **postgres_conn_id** : É o identificador da conexão com o banco de dados PostgreSQL que será usada para executar a consulta SQL. Essa conexão deve estar previamente configurada no Airflow. No exemplo, o postgres_conn_id é definido como 'LOGDW', que é o nome da conexão configurada.

* **sql** : É o SQL que será executado pelo operador. Neste caso, a consulta SQL é **"TRUNCATE TABLE varejo.TB_FATO CASCADE",** o que significa que a tarefa executará uma operação TRUNCATE na tabela TB_FATO do esquema varejo.

* **CASCADE** :  indica que qualquer dependência da tabela também será truncada.

* **dag** : É o DAG ao qual esta tarefa pertence. Um DAG é um fluxo de trabalho composto por tarefas interconectadas. A tarefa que está sendo definida aqui faz parte do DAG chamado dag_log_solutions.

Em resumo, o código cria uma tarefa chamada **'tarefa_trunca_tb_fato'** que executa uma **consulta SQL** de truncamento em uma tabela específica em um banco de dados **PostgreSQL**. Essa tarefa pode ser adicionada a um fluxo de trabalho (DAG) e programada para ser executada conforme necessário dentro do fluxo de trabalho.

---

**Tópico: Limpando as demais Tabelas**

O mesmo processo que realizamos acima foi replicado para demais tabelas do DW antes de receberem os dados.

**Tópico: Limpando a tabela DIM_CLIENTE**

In [ ]:
tarefa_trunca_dim_cliente = PostgresOperator(task_id = 'tarefa_trunca_dim_cliente', postgres_conn_id = 'LOGDW',
sql = "TRUNCATE TABLE varejo.DIM_CLIENTE CASCADE", dag = dag_log_solutions)


**Tópico: Limpando a tabela DIM_PAGAMENTO**

In [ ]:
tarefa_trunca_dim_cliente = PostgresOperator(task_id = 'tarefa_trunca_dim_cliente', postgres_conn_id = 'LOGDW',
sql = "TRUNCATE TABLE varejo.DIM_CLIENTE CASCADE", dag = dag_log_solutions)

**Tópico: Limpando a tabela DIM_FRETE**

In [ ]:
tarefa_trunca_dim_frete = PostgresOperator(task_id = 'tarefa_trunca_dim_frete', postgres_conn_id = 'LOGDW',
sql = "TRUNCATE TABLE varejo.DIM_FRETE CASCADE", dag = dag_log_solutions)

**Tópico: Limpando a tabela DIM_DATA**

In [ ]:
tarefa_trunca_dim_data = PostgresOperator(task_id = 'tarefa_trunca_dim_data', postgres_conn_id = 'LOGDW',
sql = "TRUNCATE TABLE varejo.DIM_DATA CASCADE", dag = dag_log_solutions)

**Tópico: Limpando a tabela DIM_TRANSPORTADORA**

In [ ]:
tarefa_trunca_dim_transportadora = PostgresOperator(task_id = 'tarefa_trunca_dim_transportadora', postgres_conn_id = 'LOGDW'
sql = "TRUNCATE TABLE varejo.DIM_TRANSPORTADORA CASCADE", dag = dag_log_solutions)

**Tópico: Limpando a tabela DIM_ENTREGA**

In [ ]:
tarefa_trunca_dim_entrega = PostgresOperator(task_id = 'tarefa_trunca_dim_entrega', postgres_conn_id = 'LOGDW',
sql = "TRUNCATE TABLE varejo.DIM_ENTREGA CASCADE", dag = dag_log_solutions)

**Tópico: Limpando a tabela DIM_DEPOSITO**

In [ ]:
tarefa_trunca_dim_deposito = PostgresOperator(task_id = 'tarefa_trunca_dim_deposito', postgres_conn_id = 'LOGDW',
sql = "TRUNCATE TABLE varejo.DIM_DEPOSITO CASCADE", dag = dag_log_solutions)

----------------

**Tópico criando o UpStream:**

**"Upstream"** se refere às tarefas que precedem outras tarefas no fluxo de execução.

O codigo abaixo é uma estrutura de dependência entre tarefas que é usada para garantir que as tarefas sejam executadas na ordem correta, com base nas dependências definidas.

Portanto, as tarefas **upstream** devem ser concluídas antes que as tarefas **downstream** possam ser iniciadas. É uma maneira de controlar a execução sequencial de tarefas em um fluxo de trabalho.

* **tarefa_trunca_tb_fato >>**: Esta parte indica que a tarefa chamada "tarefa_trunca_tb_fato" é uma tarefa upstream, ou seja, é uma tarefa que deve ser executada antes das tarefas que a seguem no fluxo.

* **tarefa_trunca_dim_cliente >>**: Isso significa que a tarefa "tarefa_trunca_dim_cliente" é outra tarefa upstream que deve ser executada após a conclusão da tarefa "tarefa_trunca_tb_fato".

**O mesmo padrão é repetido para as demais tarefas, onde cada uma delas depende da conclusão das tarefas upstream anteriores.**

In [ ]:
# Upstream

tarefa_trunca_tb_fato >> tarefa_trunca_dim_cliente >>

tarefa_trunca_dim_pagamento >> tarefa_trunca_dim_frete >>

tarefa_trunca_dim_data >> tarefa_trunca_dim_transportadora >>

tarefa_trunca_dim_entrega >> tarefa_trunca_dim_deposito >>

tarefa_carrega_dados_clientes >> tarefa_carrega_dados_transportadora >>

tarefa_carrega_dados_deposito >> tarefa_carrega_dados_entrega >>

tarefa_carrega_dados_frete >> tarefa_carrega_dados_pagamento >>

tarefa_carrega_dados_data >> tarefa_carrega_dados_fatos


---

###**1.6- Tópico: Bloco main**

é usado para executar um bloco de código apenas se o arquivo estiver sendo executado como um programa principal. O código dentro do bloco if só será executado se o arquivo for executado diretamente da linha de comando ou se for chamado pelo interpretador Python.

Se o arquivo for importado como um módulo, o código dentro do bloco if não será executado.

In [ ]:
# Bloco main
if __name__ == "__main__":
    dag_log_solutions.cli()



* **if name == "main"** : Esta instrução condicional verifica se o arquivo está sendo executado como um programa principal. Se for, o código dentro do bloco if será executado.

* **dag_log_solutions.cli()** : Esta função chama a função cli() no módulo dag_log_solutions. **A função cli()** é responsável por executar a interface de linha de comando do aplicativo.

---


##**Sprint 02 - Finalizada**

O processo de ETL estava completo, implementado e funcionando. Isso permitiu que os dados fossem coletados, transformados e carregados no Data Warehouse de forma automatizada e confiável.